In [146]:
from custom_fuzzy_ops import ImpliesReichenbachSigmoidal
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
import ltn

import torch
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

import globals_ltn

In [147]:
def define_connectives(connective_dict):
    # Define the fuzzy logic operators from the LTN library
    Not = connective_dict["Not"]
    And = connective_dict["And"]
    Or = connective_dict["Or"]
    Implies = connective_dict["Implies"]
    Equiv = ltn.Connective(ltn.fuzzy_ops.Equiv(And.connective_op, Implies.connective_op))
    return Not, And, Or, Implies, Equiv

Not, And, Or, Implies, Equiv = define_connectives(globals_ltn.Luk)

In [148]:
# Define a simple MLP
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(SimpleMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU()
        )
        for i in range(1, len(hidden_sizes)):
            self.model.add_module(f'fc{i}', nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            self.model.add_module(f'relu{i}', nn.ReLU())
        self.model.add_module('fc_out', nn.Linear(hidden_sizes[-1], output_size))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.model(x))
    
class flip(nn.Module):
    def __init__(self):
        super(flip, self).__init__()

    
    def forward(self, x):
        return -x

In [149]:

input_size = 2
hidden_size = [64, 64, 32, 32, 16, 16]
output_size = 1

n_objects = 100
constants_tensor_single = torch.linspace(-10, 10, n_objects)
constants_tensor = torch.cartesian_prod(constants_tensor_single, constants_tensor_single)
x1 = ltn.Variable("x1", constants_tensor)


pos = 20 * torch.rand(25, input_size) - 10
neg = 20 * torch.rand(25, input_size) - 10

pos_i = ((pos + 10) // (constants_tensor_single[1] - constants_tensor_single[0])).int()
neg_i = ((neg + 10) // (constants_tensor_single[1] - constants_tensor_single[0])).int()

In [150]:
epochs = 1000
lr = 0.0005
model1 = SimpleMLP(input_size, hidden_size, output_size)
P1 = ltn.Predicate(model1)
model2 = SimpleMLP(input_size, hidden_size, output_size)
P2 = ltn.Predicate(model2)
optimizer = torch.optim.Adam(list(model1.parameters()) + list(model2.parameters()), lr=lr)
sat_agg =ltn.fuzzy_ops.SatAgg(ltn.fuzzy_ops.AggregPMeanError(p=1))
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=1), quantifier="f")

preds1 = []
preds2 = []

with torch.no_grad():
    preds1.append(P1(x1).value.view(n_objects, n_objects))
    preds2.append(P2(x1).value.view(n_objects, n_objects))

for epoch in range(epochs):
        optimizer.zero_grad()

        x1 = ltn.Variable("x1", constants_tensor)
        x_pos = ltn.Variable("x_pos", pos)
        x_neg = ltn.Variable("x_neg", neg)

        fuzzy_theory = []

        axiom_symm = Forall([x1], Equiv(P1(x1), P2(x1)))
        fuzzy_theory.append(axiom_symm)
        
        axiom_pos = Forall([x_pos], P1(x_pos))
        fuzzy_theory.append(axiom_pos)
        # axiom_pos = Forall([x_pos], P(rho_in(x_pos)))
        # fuzzy_theory.append(axiom_pos)

        axiom_neg = Forall([x_neg], Not(P1(x_neg)))
        fuzzy_theory.append(axiom_neg)
        # axiom_neg = Forall([x_neg], Not(P(rho_in(x_neg))))
        # fuzzy_theory.append(axiom_neg)

        axiom_pos = Forall([x_pos], P2(x_pos))
        fuzzy_theory.append(axiom_pos)
        # axiom_pos = Forall([x_pos], P(rho_in(x_pos)))
        # fuzzy_theory.append(axiom_pos)

        axiom_neg = Forall([x_neg], Not(P2(x_neg)))
        fuzzy_theory.append(axiom_neg)
        # axiom_neg = Forall([x_neg], Not(P(rho_in(x_neg))))
        # fuzzy_theory.append(axiom_neg)
        

        # axiom_asym = P(x_asym)
        # fuzzy_theory.append(axiom_asym)
        # axiom_asym = Not(P(rho_in(x_asym)))
        # fuzzy_theory.append(axiom_asym)



        satisfiablity = sat_agg(*fuzzy_theory)


        loss = 1. - satisfiablity
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            preds1.append(P1(x1).value.view(n_objects, n_objects))
            preds2.append(P2(x1).value.view(n_objects, n_objects))


In [151]:
from utils_order import plot_mp

In [152]:
plot_mp(preds1, pos_examples=pos_i, neg_examples=neg_i)

interactive(children=(IntSlider(value=0, description='Frame', max=1000), Output()), _dom_classes=('widget-inte…

In [153]:
plot_mp(preds2, pos_examples=pos_i, neg_examples=neg_i)

interactive(children=(IntSlider(value=0, description='Frame', max=1000), Output()), _dom_classes=('widget-inte…

In [154]:
assert False

AssertionError: 

In [4]:
input = torch.Tensor([-2, 5]).unsqueeze(0)
input_p = torch.Tensor([5, -2]).unsqueeze(0)
W1 = nn.Linear(2, 3)
relu = nn.ReLU()
W2 = nn.Linear(3, 1)
ann = nn.Sequential(W1, relu, W2)

In [5]:
input_size = 1
hidden_size = [64,64,64,64,64,64]
output_size = 1
rho_in = ltn.Function(flip())

constants_tensor = torch.linspace(-100, 100, 1000)
x1 = ltn.Variable("x1", constants_tensor)

# pos = 20 * torch.rand(5) - 10
# neg = 20 * torch.rand(5) - 10

pos = 100 * torch.rand(3)
neg = 100 * torch.rand(3)

asym = torch.Tensor([3])

In [6]:
epochs = 1000
lr = 0.0001
model = SimpleMLP(input_size, hidden_size, output_size)
P = ltn.Predicate(model)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
sat_agg =ltn.fuzzy_ops.SatAgg(ltn.fuzzy_ops.AggregPMeanError(p=1))
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=1), quantifier="f")

preds = []

with torch.no_grad():
    preds.append(P(x1).value)

for epoch in range(epochs):
        optimizer.zero_grad()

        x1 = ltn.Variable("x1", constants_tensor)
        x_pos = ltn.Variable("x_pos", pos)
        x_neg = ltn.Variable("x_neg", neg)
        x_asym = ltn.Constant(asym)

        fuzzy_theory = []

        # axiom_symm = Forall([x1], Equiv(P(x1), P(rho_in(x1))))
        # fuzzy_theory.append(axiom_symm)
        
        axiom_pos = Forall([x_pos], P(x_pos))
        fuzzy_theory.append(axiom_pos)
        # axiom_pos = Forall([x_pos], P(rho_in(x_pos)))
        # fuzzy_theory.append(axiom_pos)

        axiom_neg = Forall([x_neg], Not(P(x_neg)))
        fuzzy_theory.append(axiom_neg)
        # axiom_neg = Forall([x_neg], Not(P(rho_in(x_neg))))
        # fuzzy_theory.append(axiom_neg)
        

        # axiom_asym = P(x_asym)
        # fuzzy_theory.append(axiom_asym)
        # axiom_asym = Not(P(rho_in(x_asym)))
        # fuzzy_theory.append(axiom_asym)



        satisfiablity = sat_agg(*fuzzy_theory)


        loss = 1. - satisfiablity
        loss.backward()
        optimizer.step()
        with torch.no_grad():
                preds.append(P(x1).value)

In [7]:
# Create sample data
tensors = preds
x_values = x1.value

num_tensors = len(tensors)
# Define the update function
def update_plot(tensor_index):
    plt.plot(x_values, tensors[tensor_index])
    plt.scatter(pos, torch.ones_like(pos), c='g')
    plt.scatter(neg, torch.zeros_like(neg), c='r')
    plt.title(f"Tensor Index {tensor_index}")
    plt.ylim(-.05, 1.05)
    # Show the updated plot
    plt.show(block=False)  # Prevent blocking behavior (optional)
    plt.show()  

slider = IntSlider(min=0, max=num_tensors - 1, step=1, description="Tensor Index")
interact(update_plot, tensor_index=slider)  # Empty function to display the plot
plt.show()  

interactive(children=(IntSlider(value=0, description='Tensor Index', max=1000), Output()), _dom_classes=('widg…